In [1]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import yaml
import os
from pathlib import Path

In [3]:
def expectativa_indicadores(canal=None):
    """
    Essa função tem o objetivo de realizar os testes que são:
    
    Comparar a média geral com todos os registros, e caso algum registro fique abaixo ou acima da regra passada no arquivo yaml, será criado uma mensagem de aviso.
        
    Parameters
    ----------
    canal: Caso tenha interesse de filtrar o DF para realizar o teste em um caso específico.

    Returns
    ----------
    bool: Um objeto booleano, sendo possível um retorno True, que significa que existe algum registro fora do esperado, e false, que significa que está dentro do esperado.
    """    
    alerts = []    
    df = pd.read_excel(os.path.join(os.getcwd(),'template_dados_fake.xlsx'))
    # Se um canal específico foi fornecido, filtra o DataFrame para esse canal
    if canal is not None:
        df = df[df['canal'] == canal]    
    
    # Convertendo a coluna 'dia' para datetime
    df['dia'] = pd.to_datetime(df['dia'])

    # Extrair o nome do dia da semana em português
    df['dia_semana'] = df['dia'].dt.strftime('%A').map({
        'Monday': 'Segunda-feira',
        'Tuesday': 'Terça-feira',
        'Wednesday': 'Quarta-feira',
        'Thursday': 'Quinta-feira',
        'Friday': 'Sexta-feira',
        'Saturday': 'Sábado',
        'Sunday': 'Domingo'
    })
    
    media = df.select_dtypes(include=['int64', 'float64']).mean()  # Calcule a média de cada coluna, excluindo a coluna 'data' e 'canal'
    
    with open(os.path.join(os.getcwd(), 'utils', 'yaml', 'expectativa_indicadores.yaml'), 'r') as file:
        regras_personalizadas = yaml.safe_load(file)

    # Lista para armazenar os alerts
    alerts = []
    
    # Percorrer as colunas e os valores
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            valores_coluna = df[col]
            media_coluna = media[col]
            
            # Verificar se a coluna tem uma regra personalizada definida
            if col in regras_personalizadas['colunas']:
                limite_inferior = regras_personalizadas['colunas'][col].get("limite_min", None)
                limite_superior = regras_personalizadas['colunas'][col].get("limite_max", None)
                media_dia_semana = regras_personalizadas['colunas'][col].get("media_dia_semana", False)
            else:
                # Se não houver regra personalizada, use os valores padrão
                limite_inferior = None
                limite_superior = None
                media_dia_semana = False
    
            # Lista para armazenar os alerts para esta coluna
            col_alerts = []
    
            # Verifique se cada valor está acima de limite_superior ou abaixo de limite_inferior da média.
            for indice, valor in enumerate(valores_coluna):
                if media_dia_semana:
                    # Calcular a média dos valores para o dia da semana atual
                    dia_semana_atual = df["dia_semana"].iloc[indice]
                    media_dia_semana_atual = df[df['dia_semana'] == dia_semana_atual][col].mean()
                    
                    if valor > media_dia_semana_atual * limite_superior or valor < media_dia_semana_atual * limite_inferior:
                        diferenca_percentual = ((valor - media_dia_semana_atual) / media_dia_semana_atual) * 100
                        alert_message = f'> • Em {df["dia"].iloc[indice].strftime("%Y-%m-%d")}, o valor da coluna {col} está {diferenca_percentual:.2f}% ({valor}) comparando com a média de todas às {dia_semana_atual} ({round(media_dia_semana_atual,2)})'
                        col_alerts.append(alert_message)
                else:
                    if limite_inferior is not None and limite_superior is not None:
                        if valor > media_coluna * limite_superior or valor < media_coluna * limite_inferior:
                            diferenca_percentual = ((valor - media_coluna) / media_coluna) * 100
                            alert_message = f'> • Em {df["dia"].iloc[indice].strftime("%Y-%m-%d")}, o valor da coluna {col} está {diferenca_percentual:.2f}% ({valor}) comparando com a média acumulada ({round(media_coluna, 2)})'
                            col_alerts.append(alert_message)
                    else:
                        # Se não houver limite definido, não faz a comparação
                        pass
                    
            # Se houver alerts para esta coluna, adicione o cabeçalho e os alerts à lista principal de alerts
            if col_alerts:
                alerts.append(f'\n> Valores para {col}:')
                alerts.extend(col_alerts)

    if not alerts:
    # Se não houver alerts, retorna False para que o ShortCircuitOperator interrompa a execução.
        print('LOG- Nenhum comportamento fora do padrão encontrado.')
        return False
    
    # Mensagem do Slack
    message = "\n".join([
        "\n",
        f":warning: *EXPECTATIVA DE INDICADORES* :warning: ",
        "\n",
        "Esse aviso pode gerar demandas no canal de #data-nation-help. Segue mais detalhes abaixo:",
        "\n",
        "> *MONITORAMENTO*: Expectativa de indicadores",
        "> *TIME RESPONSÁVEL*: @data_nation_ae_digital_sales",
        "> *OUTPUT ABAIXO* :arrow_down:",
        "\n".join([alert for alert in alerts]),
        "LINK DE COMO ATUAR:  CLIQUE AQUI",
        "LINK DA DOCUMENTAÇÃO:  CLIQUE AQUI",
        "CANAL: <!here>",
    ])
    print(message)


In [4]:
teste_a = expectativa_indicadores(canal='TESTE A')



:warning: *EXPECTATIVA DE INDICADORES* :warning: 


Esse aviso pode gerar demandas no canal de #data-nation-help. Segue mais detalhes abaixo:


> *MONITORAMENTO*: Expectativa de indicadores
> *TIME RESPONSÁVEL*: @data_nation_ae_digital_sales
> *OUTPUT ABAIXO* :arrow_down:

> Valores para coluna_a:
> • Em 2024-03-10, o valor da coluna coluna_a está 15.59% (39.5) comparando com a média de todas às Domingo (34.17)
> • Em 2024-03-24, o valor da coluna coluna_a está -14.55% (29.2) comparando com a média de todas às Domingo (34.17)
> • Em 2024-03-29, o valor da coluna coluna_a está -51.88% (27.4) comparando com a média de todas às Sexta-feira (56.94)
> • Em 2024-03-31, o valor da coluna coluna_a está -20.40% (27.2) comparando com a média de todas às Domingo (34.17)
> • Em 2024-04-05, o valor da coluna coluna_a está 16.45% (66.3) comparando com a média de todas às Sexta-feira (56.94)
> • Em 2024-04-10, o valor da coluna coluna_a está 14.78% (61.2) comparando com a média de todas às Quarta-f

In [5]:
teste_b = expectativa_indicadores(canal='TESTE B')



:warning: *EXPECTATIVA DE INDICADORES* :warning: 


Esse aviso pode gerar demandas no canal de #data-nation-help. Segue mais detalhes abaixo:


> *MONITORAMENTO*: Expectativa de indicadores
> *TIME RESPONSÁVEL*: @data_nation_ae_digital_sales
> *OUTPUT ABAIXO* :arrow_down:

> Valores para coluna_a:
> • Em 2024-03-10, o valor da coluna coluna_a está 19.50% (21.0) comparando com a média de todas às Domingo (17.57)
> • Em 2024-03-11, o valor da coluna coluna_a está 25.48% (47.5) comparando com a média de todas às Segunda-feira (37.85)
> • Em 2024-03-14, o valor da coluna coluna_a está 16.57% (40.1) comparando com a média de todas às Quinta-feira (34.4)
> • Em 2024-03-15, o valor da coluna coluna_a está 39.37% (43.9) comparando com a média de todas às Sexta-feira (31.5)
> • Em 2024-03-16, o valor da coluna coluna_a está 21.15% (22.6) comparando com a média de todas às Sábado (18.65)
> • Em 2024-03-17, o valor da coluna coluna_a está 18.37% (20.8) comparando com a média de todas às Domingo

In [6]:
teste_c = expectativa_indicadores(canal='TESTE C')

LOG- Nenhum comportamento fora do padrão encontrado.
